<a href="https://colab.research.google.com/github/anjaliprakash1512/Anjali-Prakash/blob/master/Political_Media_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"

os.environ["TF_DISABLE_JIT"] = "1"

In [3]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("JIT enabled (should be False):", tf.config.optimizer.get_jit())
print("GPUs:", tf.config.list_physical_devices("GPU"))

TF version: 2.19.0
JIT enabled (should be False): 
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


**IMPORTING SOME NECESSARY LIBRARIES**

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.compose import ColumnTransformer
import tensorflow as tf
import keras_nlp
import pandas as pd
from sklearn.model_selection import train_test_split

**IMPORTING DATASET**

In [6]:
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Opens file chooser
for file_name in uploaded.keys():
    print(f"Uploaded: {file_name}")
    data = pd.read_csv(file_name, encoding='latin1')

data.head()


Saving political_social_media.csv.zip to political_social_media.csv.zip
Uploaded: political_social_media.csv.zip


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,...,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,...,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,...,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,...,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


**KEEPING RELEVANT COLUMNS**

In [7]:
data = data[['text', 'bias']].dropna()

# Inspect label distribution
print(data['bias'].value_counts())

bias
neutral     3689
partisan    1311
Name: count, dtype: int64


**DATA CLEANING**

In [8]:
import re
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)
    return text.strip()

data['clean_text'] = data['text'].apply(clean_text)
data.head()


,text,bias,clean_text
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,partisan,RT : Rep. Trey Radel (R- FL) slams Obamacare. ...
1,VIDEO - #Obamacare: Full of Higher Costs and ...,partisan,VIDEO - Obamacare: Full of Higher Costs and B...
2,Please join me today in remembering our fallen...,neutral,Please join me today in remembering our fallen...
3,RT @SenatorLeahy: 1st step toward Senate debat...,neutral,RT : 1st step toward Senate debate on Leahy-Cr...
4,.@amazon delivery #drones show need to update ...,partisan,. delivery drones show need to update law to p...


In [9]:
data['label'] = data['bias'].map({'neutral': 0, 'partisan': 1})

In [10]:
from sklearn.utils import resample

majority_class = data[data.label == 0]
minority_class = data[data.label == 1]

minority_oversampled = resample(minority_class,
                                 replace=True,
                                 n_samples=len(majority_class),
                                 random_state=42)

data = pd.concat([majority_class, minority_oversampled])

In [11]:
print(data['label'].value_counts())

label
0    3689
1    3689
Name: count, dtype: int64


**BERT MODEL**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    data['clean_text'], data['label'], test_size=0.2, random_state=42, stratify=data['bias'])

train_texts = X_train.tolist()
test_texts = X_test.tolist()
train_labels = y_train.astype(int).tolist()
test_labels = y_test.astype(int).tolist()

train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).shuffle(1000).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels)).batch(16)


In [14]:
model2 = keras_nlp.models.BertClassifier.from_preset("bert_base_en_uncased", num_classes=2)

In [15]:
model2.compile(
    optimizer=tf.keras.optimizers.Adam(3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [16]:
history = model2.fit(train_ds, validation_data=test_ds, epochs=3)

Epoch 1/3
369/369 ━━━━━━━━━━━━━━━━━━━━ 696s 2s/step - accuracy: 0.7350 - loss: 0.5519 - val_accuracy: 0.8178 - val_loss: 0.3959
Epoch 2/3
369/369 ━━━━━━━━━━━━━━━━━━━━ 575s 2s/step - accuracy: 0.8679 - loss: 0.3196 - val_accuracy: 0.8767 - val_loss: 0.2931
Epoch 3/3
369/369 ━━━━━━━━━━━━━━━━━━━━ 575s 2s/step - accuracy: 0.9504 - loss: 0.1406 - val_accuracy: 0.8957 - val_loss: 0.2944


In [17]:
model2.evaluate(test_ds)

93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 512ms/step - accuracy: 0.8971 - loss: 0.2714


[0.294366717338562, 0.8956639766693115]

**PREDICTING DATA FOR UNSEEN ARTICLES**

In [18]:
samples = [
    # 1. Expected: partisan (negative opinion)
    "The proposed tax hike is a disastrous move that will crush small businesses and cripple the economy.",

    # 2. Expected: neutral (factual report)
    "The bill requires companies with over 50 employees to offer healthcare benefits.",

    # 3. Expected: partisan (positive spin)
    "This brilliant and compassionate legislation is a landmark victory for working families across the nation.",

    # 4. Expected: neutral (diplomatic update)
    "Diplomats from both countries met for three hours to discuss the ongoing border dispute.",

    # 5. Expected: partisan (fear-based language)
    "Failing to pass this bill will leave our nation vulnerable and endanger our security.",

    # 6. Expected: neutral (statistical report)
    "According to the Department of Commerce, retail sales increased by 0.5% in the last quarter.",

    # 7. Expected: partisan (unverified claim)
    "It's clear the opposition's entire platform is built on nothing but lies and misinformation."
]

In [19]:
preds = model2.predict(samples)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


In [20]:
preds

array([[-2.2368913 ,  3.2820415 ],
       [ 0.18402053,  0.25826287],
       [ 1.7326285 , -1.7729362 ],
       [ 1.6825111 , -2.1935768 ],
       [ 0.08285493,  0.7155641 ],
       [ 2.0223417 , -2.0842748 ],
       [-0.67212063,  1.4853464 ]], dtype=float32)

In [21]:
labels = tf.argmax(preds, axis=1).numpy()

In [22]:
labels

array([1, 1, 0, 0, 1, 0, 1])

In [23]:
for text, label in zip(samples, labels):
    print(f"{text} -> {'partisan' if label==1 else 'neutral'}")

The proposed tax hike is a disastrous move that will crush small businesses and cripple the economy. -> partisan
The bill requires companies with over 50 employees to offer healthcare benefits. -> partisan
This brilliant and compassionate legislation is a landmark victory for working families across the nation. -> neutral
Diplomats from both countries met for three hours to discuss the ongoing border dispute. -> neutral
Failing to pass this bill will leave our nation vulnerable and endanger our security. -> partisan
According to the Department of Commerce, retail sales increased by 0.5% in the last quarter. -> neutral
It's clear the opposition's entire platform is built on nothing but lies and misinformation. -> partisan


**FINAL ACCURACY - 90%**